In [18]:
import torch
from torch import nn
import numpy as np
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [2]:
!rm -rf torch_compile_debug

In [20]:
# def fn(x):
#     return x**2 + 2*x + 1

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)
        
    def forward(self, x):
        return self.fc2(self.fc1(x))

In [21]:
fn = Model().to(device)

fn_compiled = torch.compile(fn, backend="inductor", 
                       options={'trace.graph_diagram':True,
                                'trace.enabled':True})
x = torch.rand(10)
out = fn_compiled(x.to(device)).sum().backward()

[2023-03-21 07:38:43,245] torch._inductor.debug: [WARNING] model__5_forward_16 debug trace: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_21_07_29_30_778184-pid_24890/aot_torchinductor/model__5_forward_16.10
[2023-03-21 07:38:43,265] torch._inductor.debug: [WARNING] model__5_backward_17 debug trace: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_21_07_29_30_778184-pid_24890/aot_torchinductor/model__5_backward_17.11


Writing FX graph to file: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_21_07_29_30_778184-pid_24890/aot_torchinductor/model__5_forward_16.10/graph_diagram.svg
Writing FX graph to file: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_21_07_29_30_778184-pid_24890/aot_torchinductor/model__5_backward_17.11/graph_diagram.svg


In [9]:
from torch.fx import passes, symbolic_trace
model = symbolic_trace(fn)

g = passes.graph_drawer.FxGraphDrawer(model, 'fn')
with open("unoptimized_graph1.svg", "wb") as f:
    f.write(g.get_dot_graph().create_svg())
    
print(model.code)




def forward(self, x):
    pow_1 = x ** 2
    mul = 2 * x;  x = None
    add = pow_1 + mul;  pow_1 = mul = None
    add_1 = add + 1;  add = None
    return add_1
    
